In [1]:
import sys
import numpy as np
import numpy as np
from scipy.io import wavfile
from matplotlib import pyplot as plt, patches
from matplotlib.lines import Line2D
import pandas as pd
import math

# Hardcoded path to the directory you want to import from
path_to_parent = 'c:/Users/Alex/source/repos/COMP5400M-CW2'
sys.path.insert(0, path_to_parent)

from doa import DoaCNN

ROOM_DIM = [10.,10.]
AUDIO_PATH = 'c:/Users/Alex/source/repos/COMP5400M-CW2/sound_data/cricket.wav'

In [ ]:
microphones = np.c_[[ROOM_DIM[0]/2-0.1, 0.5, 0], [ROOM_DIM[0]/2+0.1, 0.5, 0]]
source_loc = [[ROOM_DIM[0]/2+2, ROOM_DIM[1]-1], [ROOM_DIM[0]/2-3, ROOM_DIM[1]-2]]
centre_mic = [ROOM_DIM[0]/2, 0.5]
simulation_done = False

In [ ]:
def process(source, mic, ax, pred):
    new_mic = [0, 0]
    if (source[0]-0.3 < mic[0] < source[0]+0.3 and 
        source[1]-0.3 < mic[1] < source[1]+0.3):
        simulation_done = True
    elif (0.3 > mic[0] > 5 or 0.3 > mic[1] > 5):
        simulation_done = True
    else:
        new_mic = update_centre(mic, ax, pred)

    return new_mic


def update_centre(mic, ax, pred):
    pred_degree = max(set(pred), key=pred.count)
    pred_rad = math.pi - pred_degree*math.pi/180

    x_align = mic[0] + 0.3*np.cos(pred_rad)
    y_align = mic[1] + 0.3*np.sin(pred_rad)
    new_mic = [0, 0]

    if not x_align == False:
        ax.add_patch(patches.FancyArrowPatch((float(mic[0])+0.01, float(mic[1]+0.01)), 
                                            (float(x_align)-0.01, float(y_align)-0.01), 
                                            mutation_scale=20,
                                            alpha = 0.8, arrowstyle='->'))
        ax.add_patch(patches.Circle((mic[0], mic[1]), radius=0.1, edgecolor='red', facecolor='none', linewidth=2))
        
        new_mic = [x_align, y_align]

    return new_mic

In [ ]:
# Generate room
plt.rcParams.update({'figure.figsize': [8,8], 'figure.autolayout': True, 'font.size': 16})
fig, _ = plt.subplots()
ax = plt.gca()
ax.set_xlim([0, ROOM_DIM[0]+0.5])
ax.set_ylim([0, ROOM_DIM[1]+0.5])

# Include sound source patch
sound_1 = patches.Circle((float(source_loc[0][0]), float(source_loc[0][1])), radius=0.1, facecolor='green', linewidth=5, label='Sound Source')
sound_2 = patches.Circle((float(source_loc[1][0]), float(source_loc[1][1])), radius=0.1, facecolor='green', linewidth=5, label='Sound Source')
ax.add_patch(sound_1)
ax.add_patch(sound_2)

# Include custom legend
legend_elements = [Line2D([0], [0], marker='o', lw=2, color='green',   label='Sound Source'),\
                   Line2D([0], [0], marker='o', lw=2, color="#ff8f0f", label='Cricket')]
ax.legend(handles=legend_elements, loc='upper left')

# Generate room
fs, signal = wavfile.read(AUDIO_PATH)
i = 0

# Generate predictions
while simulation_done == False:
    '''
        snr factor:
        - 0.1   -> SNR = 10dB
        - 0.01  -> SNR = 20dB
        - 0.001 -> SNR = 30dB
    '''
    CNN = DoaCNN(ROOM_DIM, source_loc, centre_mic, 0.01)
    CNN.get_room(signal)

    # Localize sound
    pred = CNN.get_prediction()
    centre_mic = process(source_loc[0], centre_mic, ax, pred)
    
    i += 1
    plt.savefig("path/path_ideal_%02d.png"%i)
    plt.show